## Not working code

In [1]:
import pandas as pd
import numpy as np
import math
from transformers import AutoTokenizer, AutoModel
import torch

In [3]:
# Init CodeBERT
tokenizer_bert = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model_bert = AutoModel.from_pretrained("microsoft/codebert-base")

In [4]:
# DO NOT forget 'keep_default_na=False' --> otherwise some NaN values in read data
df = pd.read_csv('./big-vul_dataset/line_sample_20p_balanced_ratio.csv', skipinitialspace=True, low_memory=True, keep_default_na=False)

In [ ]:
from sklearn.model_selection import train_test_split

# split into training and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(df['line'], df['vul'], test_size=0.2, random_state=42)

print('X_tr shape:',X_tr.shape)

In [14]:
def bert_encode(list_of_strings):
    embeddings = []
    for s in list_of_strings:
        # tokenize
        code_tokens=tokenizer_bert.tokenize(s)
        # add special tokens
        tokens=[tokenizer_bert.cls_token]+code_tokens+[tokenizer_bert.sep_token]
        # convert to IDs
        tokens_ids=tokenizer_bert.convert_tokens_to_ids(tokens)
        print(tokens_ids)
        # create embedding
        context_embeddings=model_bert(torch.tensor(tokens_ids)[None,:])[0]
        # reforming tensor into numpy array (check needed!)
        embeddings.append(context_embeddings)
 
    return embeddings

In [15]:
x = bert_encode(["int x = 5"])
x[0].shape

[0, 2544, 3023, 5457, 195, 2]


torch.Size([1, 6, 768])

In [16]:
x = bert_encode(["intint"])
x

[0, 2544, 2544, 2]


[tensor([[[-0.1278,  0.3240,  0.0276,  ..., -0.1703, -0.2517,  0.3435],
          [-0.5561,  0.4523, -0.0856,  ..., -0.1599, -0.4564,  0.1024],
          [-0.6194,  0.5343, -0.1553,  ..., -0.4806, -0.2627,  0.1478],
          [-0.1275,  0.3236,  0.0281,  ..., -0.1705, -0.2517,  0.3432]]],
        grad_fn=<NativeLayerNormBackward0>)]

In [ ]:
# define how big a batch of entries should be (depending on RAM)
batch_size = 100

# number of epochs is calulated based on the batch_size
epochs = math.ceil(len(X_tr)/batch_size)

# split the dataframes (X_tr, y_tr) into an array of dataframes (number of epochs)
batchesX = np.array_split(X_tr, epochs)
batchesY = np.array_split(y_tr, epochs)

In [ ]:
# import and initialisation of generic MLPClassifier
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1)

# iterate over the number of epochs (manually decreased the number of epochs here to ensure fast processing)
for i in range(epochs-50):
    print(i)
    # take a batch and process it and partial_fit the model to the batch
    X_batch, Y_batch = batchesX[i], batchesY[i]
    encodedBatch = bert_encode(X_batch.tolist())
    clf.partial_fit(encodedBatch, Y_batch, classes=np.unique(y_tr))

In [ ]:
from sklearn.metrics import classification_report


y_pred = clf.predict(bert_encode(X_val[:batch_size*3]))
print(classification_report(y_val[:batch_size*3], y_pred))

In [ ]:
X_tr_bert = bert_encode(X_tr.tolist())
X_val_bert = bert_encode(X_val.tolist())

In [ ]:
from sklearn.linear_model import LogisticRegression

# LR model
vul_model = LogisticRegression()
# train
vul_model = vul_model.fit(X_tr_bert, y_tr)
# predict
pred_vul = vul_model.predict(X_val_bert)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, pred_bert))